In [1]:
# !pip install tensorflow-gpu

In [3]:
import os
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.layers.experimental.preprocessing import Normalization
from tensorflow.keras.optimizers import Adam

from tensorflow.keras import utils
from tensorflow.keras.preprocessing import image, image_dataset_from_directory
import matplotlib.pyplot as plt
%matplotlib inline

from os import listdir
from shutil import copy2

from warnings import filterwarnings
filterwarnings('ignore')

In [4]:
# Создадим структуру директорий, потом применим предобученную нейронную сеть с помощью технологий трансфер-лернинга

### Train_dataset

In [9]:
data = pd.read_csv('butterflies/final_data/train_labels.csv')
data

,filenames,label
0,train/0.jpg,12
1,train/1.jpg,33
2,train/2.jpg,4
3,train/3.jpg,42
4,train/4.jpg,26
...,...,...
4950,train/4950.jpg,33
4951,train/4951.jpg,26
4952,train/4952.jpg,34
4953,train/4953.jpg,30


In [29]:
pd.unique(data['label'])

array([12, 33,  4, 42, 26,  0,  2, 31, 25, 18,  6, 28, 45, 40, 27, 10, 16,
       36, 37, 20, 14, 39,  5,  7,  3, 38, 15, 23, 24, 32, 30, 48, 19, 17,
        1,  8, 44, 29, 43, 22, 49, 41, 35, 34, 47, 46, 21, 13,  9, 11])

In [28]:
os.mkdir('buffer_for_learning')
os.mkdir('buffer_for_learning/train')
for i in pd.unique(data['label']):
    os.mkdir('buffer_for_learning/train/' + str(i))

In [30]:
for name, label in data.values:
    copy2('butterflies/final_data/' + name, 'buffer_for_learning/train/'+ str(label) + '/' + name.split('/')[1])

### Valid_dataset

In [31]:
valid = pd.read_csv('butterflies/final_data/valid_labels.csv')
valid

,filenames,label
0,valid/0.jpg,14
1,valid/1.jpg,40
2,valid/2.jpg,8
3,valid/3.jpg,37
4,valid/4.jpg,27
...,...,...
245,valid/245.jpg,32
246,valid/246.jpg,6
247,valid/247.jpg,2
248,valid/248.jpg,4


In [33]:
len(pd.unique(valid['label']))

50

In [34]:
os.mkdir('buffer_for_learning/valid')
for i in pd.unique(valid['label']):
    os.mkdir('buffer_for_learning/valid/' + str(i))

In [35]:
for name, label in valid.values:
    copy2('butterflies/final_data/' + name, 'buffer_for_learning/valid/'+ str(label) + '/' + name.split('/')[1])

In [5]:
train_dataset = image_dataset_from_directory('buffer_for_learning/train',
                                             seed=42,
                                             batch_size=32,
                                             image_size=(224, 224))

valid_dataset = image_dataset_from_directory('buffer_for_learning/valid',
                                             seed=42,
                                             batch_size=32,
                                             image_size=(224, 224))



Found 4955 files belonging to 50 classes.
Found 250 files belonging to 50 classes.


2021-12-19 14:22:31.571211: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-19 14:22:31.571897: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-12-19 14:22:31.571958: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2021-12-19 14:22:31.572043: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2021-12-19 14:22:31.572082: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Co

In [3]:
# train_dataset.class_names

In [6]:
vgg16_net = VGG16(weights='imagenet',
                  include_top=False, 
                  input_shape=(224, 224, 3))

In [7]:
model = Sequential()
model.add(Normalization())
model.add(vgg16_net)
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(50, activation='softmax'))

In [8]:
vgg16_net.trainable = True
trainable = False
for layer in vgg16_net.layers:
    if layer.name == 'block5_conv2':
        trainable = True
    layer.trainable = trainable

In [9]:
model.compile(loss='categorical_crossentropy',
              optimizer="adam",
              metrics=['accuracy'])

In [1]:
# history = model.fit(train_dataset, 
#                     validation_data=valid_dataset,
#                     epochs=15)

In [ ]:
# Оцениваем качество обучения модели на тестовых данных
scores = model.evaluate(valid_dataset, verbose=1)

In [ ]:
plt.plot(history.history['accuracy'], 
         label='Доля верных ответов на обучающем наборе')
plt.plot(history.history['val_accuracy'], 
         label='Доля верных ответов на проверочном наборе')
plt.xlabel('Эпоха обучения')
plt.ylabel('Доля верных ответов')
plt.legend()
plt.show()

In [ ]:
model.save("model.h5")